# **Create *soil_properties.nc* for WRF-Hydro**

In [ ]:
# Library
import netCDF4 as nc4
import numpy as np
import os
import pandas as pd
import pathlib as pl

from nco import Nco
nco = Nco()

In [ ]:
#### Input geogrid:
geoFile = 'geo_em.d01.nc'

#### Input parameter tables:
soilParamFile = 'SOILPARM.TBL'
mpParamFile = 'MPTABLE.TBL'
genParamFile = 'GENPARM.TBL'
hydParamFile = 'HYDRO.TBL'

In [ ]:
#### Output files to create: 
# IMPORTANT: The netcdf files below will be overwritten if they exist!
slpropFile = 'soil_properties.nc'
slpropFile2 = "soil_properties2.nc"
hyd2dFile = 'hydro2dtbl.nc'
hyd2dFile2 = "hydro2dtbl2.nc"

In [ ]:
# Note that if TRUE, the script will overwrite the geogrid file specified above. ## Not included in this code
updateTexture = False

#### Category to fill in for soil class if a cell is water in the soil layer but NOT water in the land cover layer:
# If the script encounters a cell that is classified as land in the land use field (LU_INDEX) but is 
# classified as a water soil type, it will replace the soil type with the value you specify below.
# If updateTexture is TRUE, these chages will be propagated to the geogrid. If not, they are just
# used in parameter assignment. 
# Ideally there are not very many of these, so you can simply choose the most common soil type in 
# your domain. Alternatively, you can set to a "bad" value (e.g., -8888) to see how many of these 
# conflicts there are. If you do this DO NOT RUN THE MODEL WITH THESE BAD VALUES. Instead, fix them 
# manually with a neighbor fill or similar fill algorithm.
soilFillVal = 3

#### Hard-wire urban soil properties in hydro 2d table?
# Some soil parameters are hard-coded to preset values in NoahMP and WRF-Hydro for urban land cover cells.
# If you want to show these in your hyd2dFile parameter file, set this to TRUE. If you want to show
# default parameters, set to FALSE. There should be no answer differences either way.
setUrban = False

## **Do not update below here**

In [ ]:
# Soil Properties
nameLookupSoil = {'smcref':"REFSMC", 'dwsat':"SATDW", 'smcdry':"DRYSMC", 'smcwlt':"WLTSMC",
                   'bexp':"'BB", 'dksat':"SATDK", 'psisat':"SATPSI", 'quartz':"QTZ",
                   'refdk':"REFDK", 'refkdt':"REFKDT", 'slope':"SLOPE", 'smcmax':"MAXSMC",
                   'cwpvt':"CWPVT", 'vcmx25':"VCMX25", 'mp':"MP", 'hvt':"HVT", 'mfsno':"MFSNO",
                   'rsurfexp':"RSURF_EXP"}
var3d = ["smcref", "dwsat", "smcdry", "smcwlt", "bexp", "dksat", "psisat", "quartz", "smcmax"]

# Hydro 2D Table
nameLookupHyd = {'SMCMAX1':"smcmax", 'SMCREF1':"smcref", 'SMCWLT1':"smcwlt", 
                   'OV_ROUGH2D':"OV_ROUGH2D", 'LKSAT':"dksat"}

In [ ]:
nco.ncks(input=geoFile, output=slpropFile, 
         options=["--ovr","--netcdf4","--variable HGT_M","--no_tmp_fl"])

In [ ]:
#### Create new soil properties file with fill values

# new 'soil_properties.nc'
ncid = nc4.Dataset(slpropFile,'r+', format='NETCDF4')

# dimensions
sndim = ncid.dimensions['south_north']
wedim = ncid.dimensions['west_east']
soildim = ncid.createDimension('soil_layers_stag', 4)
timedim = ncid.dimensions['Time']

# variables
for i in nameLookupSoil:
    if i in var3d:
        ncid.createVariable(i, 'f4', ('Time', 'soil_layers_stag', 'south_north', 'west_east'))
    else:
        ncid.createVariable(i, 'f4', ('Time', 'south_north', 'west_east')) 
ncid.close()

In [ ]:
nco.ncks(input=slpropFile, output=slpropFile2, 
         options=["--ovr","-x","--variable HGT_M","--no_tmp_fl"])

### **Read parameter tables**

In [ ]:
# SOILPARM
if os.path.exists(soilParamFile):
    soltab = pd.read_table(soilParamFile, header=None, skiprows=3, index_col=False,
                           sep=",", comment="!", skip_blank_lines=True, nrows=19)

    solhead = pd.read_table(soilParamFile, header=None, index_col=False,
                            skiprows=2, sep="\s+",nrows=1).to_numpy()[0]
    solhead[0],solhead[-1] = 'solID','solName'
    soltab.columns = solhead
else:
    print('No soil parameter file found.')

soltab = soltab.set_index('solID')

In [ ]:
# MPTABLE
if os.path.exists(mpParamFile):
    with open(mpParamFile,'r') as fin:
        lines = fin.readlines()
        mptab = pd.DataFrame()
        for ii in range(len(lines)):
            if '&noahmp_usgs_parameters' in lines[ii]:
                for line in lines[ii:]:
                    if len(line.split('='))>1:
                        if len(line.split('=')[1].strip().replace(' ','').split(',')[:-1])>20:
                            mptab[line.split('=')[0].strip()] = line.split('=')[1].strip().replace(' ','').split(',')[:-1]
                mptab['vegID'] = np.arange(1,28,1)
            elif 'RSURF_EXP' in lines[ii]:
                rsurfexp_text = lines[ii]
                rsurfexp = float(rsurfexp_text.split('=')[1].split('!')[0].strip())
                mpglobtab = {'RSURF_EXP':rsurfexp}
            else:
                pass
else:
    print('No MP parameter file found.')

mptab = mptab.set_index('vegID')

In [ ]:
# GENPARM
if os.path.exists(genParamFile):
    with open(genParamFile,'r') as fin:
        gendump = fin.readlines()
        slopeVal = float(gendump[gendump.index("SLOPE_DATA\n")+2])
        refkdtVal = float(gendump[gendump.index("REFKDT_DATA\n")+1])
        refdkVal = float(gendump[gendump.index("REFDK_DATA\n")+1])
        gentab = {'REFDK':refdkVal, 'REFKDT':refkdtVal, 'SLOPE':slopeVal}
else:
    print('No GENPARM parameter file found.')

In [ ]:
# HYDPARM
if os.path.exists(hydParamFile):
    hydtab = pd.DataFrame()
    r,des = [],[]
    with open(hydParamFile,'r') as fin:
        hydhead = fin.readlines()
    pcount = int(hydhead[0].strip().split(' ')[0])
    for i in range(2,pcount+2):
        des.append(hydhead[i].strip().split(',')[1])
        r.append(hydhead[i].strip().split(',')[0])
    hydtab["OV_ROUGH2D"] = r
    hydtab['descrip'] = des
    hydtab['vegID'] = np.arange(1,29,1)

### **Spatial soil parameter files**

In [ ]:
# Get 2D fields
if os.path.exists(geoFile):
    geoin = nc4.Dataset(geoFile,'r')
    geoin.set_auto_mask(False)
    solmap = geoin.variables['SCT_DOM'][:]
    vegmap = geoin.variables['LU_INDEX'][:]
    # get attributes
    soilWater = geoin.getncattr('ISOILWATER')
    vegWater = geoin.getncattr('ISWATER')
    maxSoilClass = geoin.dimensions['soil_cat'].size
    vegUrban = geoin.getncattr('ISURBAN')
    solmap[(vegmap != vegWater) & (solmap == soilWater)] = soilFillVal
    solmap[vegmap == vegWater] = soilWater
    geoin.close()
else:
    print('No geogrid file found.')

In [ ]:
# Get new soil props file
ncid = nc4.Dataset(slpropFile2,'r+')
ncid.set_auto_mask(False)
paramList = list(ncid.variables.keys())

# Loop through params and update
print('updating: {}'.format(slpropFile2))
for param in paramList:
    paramName = nameLookupSoil[param]
    print("Processing {}".format(param))
    #if param is not None:   
    if paramName in soltab.columns:
        ncvar = ncid.variables[param][:]
        pnew = np.copy(solmap)
        #pnew[~(pnew in soltab['solID'])] = -9999 ## see how to assign -9999 to NaN
        #pnew[pnew==soltab['solID']] = soltab[paramName] ## instead for loop
        for i in soltab.index:
            paramValue = soltab.loc[i, paramName]
            pnew[solmap==i] = paramValue
        #pnew3d = np.reshape(np.repeat(pnew, ncvar.shape[1]), ncvar.shape)
        #pnew3d[pnew3d < -9998] = ncvar[pnew3d < -9998]
        ncid.variables[param][:] = pnew        
    elif paramName in mptab.columns:
        ncvar = ncid.variables[param][:]
        pnew = np.copy(vegmap)
        #pnew[~(pnew in mptab['vegID'].values)] = -9999 ## see how to assign -9999 to NaN
        for i in mptab.index:
            paramValue = mptab.loc[i, paramName]
            pnew[vegmap==i] = paramValue
        pnew = np.reshape(np.repeat(pnew,np.shape(ncvar)[0]),ncvar.shape)
        #pnew[pnew < 0] = ncvar[pnew < 0]
        ncid.variables[param][:] = pnew            
    elif paramName in gentab:
        ncvar = ncid.variables[param][:]
        pnew = ncvar*0 + gentab[paramName]
        ncid.variables[param][:] = pnew
    elif paramName in mpglobtab:
        ncvar = ncid.variables[param][:]
        pnew = ncvar*0 + mpglobtab[paramName]
        ncid.variables[param][:] = pnew            

ncid.close()

In [ ]:
os.remove('soil_properties.nc')

In [ ]:
os.rename('soil_properties2.nc', 'soil_properties.nc')